# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from ipywidgets.embed import embed_minimal_html

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
fpath = '../output_data/NathanAsh_cities.csv'

city = pd.read_csv(fpath)
city

,Unnamed: 0,City,Latitude,Longitude,Max Tempurature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country
0,1,sola,-13.88,167.55,80.258,82.0,7,6.57,VU
1,3,vaini,-21.20,-175.20,70.844,63.0,71,6.63,TO
2,5,de-kastri,51.47,140.78,62.960,71.0,100,6.55,RU
3,7,matola,-25.96,32.46,59.000,77.0,0,4.60,MZ
4,9,porto velho,-8.76,-63.90,80.600,78.0,20,0.54,BR
...,...,...,...,...,...,...,...,...,...
489,600,takoradi,4.88,-1.76,74.984,88.0,100,3.97,GH
490,602,zvolen,48.57,19.15,64.400,88.0,100,1.28,SK
491,603,san felipe,10.34,-68.74,72.212,92.0,100,0.60,VE
492,604,boyolangu,-8.12,111.89,81.230,72.0,23,2.38,ID


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:


loc = city[['Latitude','Longitude']]
hum = city['Humidity (%)'].astype(float)


In [56]:

fig = gmaps.figure()
heatH = gmaps.heatmap_layer(loc,weights = hum, 
                           dissipating = False, max_intensity = 10,
                           point_radius = .5)
fig.add_layer(heatH)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [46]:
ideal = city.copy()
# ideal['Humidity']  = pd.to_numeric(ideal['Humidity'])

# for index,row in city_df.iterrows():
ideal = ideal.loc[ideal['Max Tempurature (F)'].gt(80) == False]
ideal = ideal.loc[ideal['Max Tempurature (F)'].gt(70) == True]
ideal = ideal.loc[ideal['Wind Speed (mph)'].gt(10) == False]
ideal = ideal.loc[ideal['Cloudiness (%)'].gt(0) == False]
ideal = ideal.dropna(how = 'any')
ideal = ideal.drop(['Unnamed: 0'], axis =1)
ideal


,City,Latitude,Longitude,Max Tempurature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country
6,atuona,-9.80,-139.03,77.900,70.0,0,7.44,PF
12,avarua,-21.21,-159.78,78.800,88.0,0,4.60,CK
38,caravelas,-17.71,-39.25,70.250,86.0,0,2.23,BR
66,komsomolskiy,40.43,71.72,75.200,36.0,0,4.10,UZ
105,hun,29.13,15.95,71.564,58.0,0,0.84,LY
116,palmas,-10.21,-48.36,77.000,57.0,0,3.10,BR
121,airai,-8.93,125.41,75.164,32.0,0,2.44,TL
142,sabha,27.04,14.43,78.116,33.0,0,6.33,LY
155,guerrero negro,27.98,-114.06,74.840,62.0,0,5.45,MX
173,tortoli,39.93,9.66,76.820,41.0,0,1.87,IT


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
#For loop 

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


ideal["Hotel Name"] = ""
ideal['Country']= ''

for index, row in ideal.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']

    params['location'] = f"{lat},{lng}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    urlpar = requests.get(url,  params=params)
    urlpar = urlpar.json()
    try:
        ideal
        ideal.loc[index, 'Hotel Name'] = urlpar['results'][0]['name']
        print(f'Hotel in {row[0]} found')
        
    except:
        print(f'Hotel in {row[0]} not found')
        
ideal
    


Hotel in atuona found
Hotel in avarua found
Hotel in caravelas found
Hotel in komsomolskiy found
Hotel in hun found
Hotel in palmas found
Hotel in airai found
Hotel in sabha found
Hotel in guerrero negro found
Hotel in tortoli found
Hotel in turayf found
Hotel in gat found
Hotel in kamariotissa found
Hotel in abu samrah not found
Hotel in nador found
Hotel in saquarema found
Hotel in abu kamal found
Hotel in marsa matruh found
Hotel in beamsville found
Hotel in lagoa found
Hotel in arrecife found
Hotel in nusaybin found
Hotel in awjilah not found
Hotel in jalu found
Hotel in uruzgan not found


,City,Latitude,Longitude,Max Tempurature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Hotel Name
6,atuona,-9.80,-139.03,77.900,70.0,0,7.44,,Villa Enata
12,avarua,-21.21,-159.78,78.800,88.0,0,4.60,,Paradise Inn
38,caravelas,-17.71,-39.25,70.250,86.0,0,2.23,,Pousada dos Navegantes
66,komsomolskiy,40.43,71.72,75.200,36.0,0,4.10,,Ikathouse
105,hun,29.13,15.95,71.564,58.0,0,0.84,,Alrwasi Hotel
116,palmas,-10.21,-48.36,77.000,57.0,0,3.10,,Hotel Lago da Palma
121,airai,-8.93,125.41,75.164,32.0,0,2.44,,Juvinal Martin
142,sabha,27.04,14.43,78.116,33.0,0,6.33,,محطة ام الارانب -سبها مواصلات
155,guerrero negro,27.98,-114.06,74.840,62.0,0,5.45,,"Casa Laguna, Bed & Breakfast"
173,tortoli,39.93,9.66,76.820,41.0,0,1.87,,Hotel La Bitta


In [63]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
hotel = ideal.copy()
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotelinfo = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel[["Latitude", "Longitude"]]

In [65]:
# Add marker layer ontop of heat map
# Display figure
marker = gmaps.marker_layer(locations,
                           hover_text = hotelinfo)
fig.add_layer(marker)
fig

Figure(layout=FigureLayout(height='420px'))